<img align="right" src="images/DSApps_logo_small.jpg">

# DSApps 2023 @ TAU: Assignment 4

### Giora Simchoni

### Network Analysis with NetworkX

### Welcome

Welcome to Assignment 4 in Python!

Remember:

* You can play with the assignment in Playground mode, but:
* Only your private Github repository assigned to you by the course admin will be cloned and graded (Submission mode, see instructions [here](https://github.com/DSApps-2023/Class_Slides/blob/main/Apps_of_DS_HW.pdf))
* Like any other University assignment, your work should remain private
* You need to `git clone` your private Github repository locally as explained [here](https://github.com/DSApps-2023/Class_Slides/blob/main/Apps_of_DS_HW.pdf)
* You need to uncomment the starter code inside the chunk, replace the `### YOUR CODE HERE ###`, run the chunk and see that you're getting the expected result
* Pay attention to what you're asked to do and the required output
* For example, using a *different* function than the one you were specifically asked to use, will decrease your score (unless you amaze me)
* Your notebook should run smoothly from start to end if someone presses in the Jupyter toolbar Kernel --> Restart & Run All
* When you're done save the entire notebook into a html file, this is the file that would be graded
* You can add other files but do not delete any files
* Commit your work and push to your private Github repository as explained [here](https://github.com/DSApps-2023/Class_Slides/blob/main/Apps_of_DS_HW.pdf)

This assignemtnt is due: 18/5 23:59

### Libraries

These are the libraries you will need. If you don't have them, you need to uncomment the `!pip install` line and install them first (you can also just copy this command to a terminal and do it there if you don't want all the output printed in this notebook).

In [ ]:
#!pip install matplotlib numpy scipy pandas networkx

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
from itertools import combinations # not mandatory

### The Films to Actors 2010s Dataset



Often cited articles in the field of Network Analysis are ["Emergence of Scaling in Random Networks"](http://barabasi.com/f/67.pdf) by Barabasi and Albert and ["The Eccentricities of Actors"](https://www.jstor.org/stable/25678151?seq=1) by Harris and Mossinghoff. Both articles deal with a network of film actors in which two actors are connected if they were cast in the same film. The famous [Kevin Bacon Number](https://en.wikipedia.org/wiki/Six_Degrees_of_Kevin_Bacon) is also discussed.

Alas, the data used by these articles were put together somewhere in the late 1990s :(

Can we scrape the web to get an updated version?

We sure can! Checkout the short python script `scraping_films_actors_2010s.py` in which I used one of Python's most popular library for scraping the web [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/), to scrape [Wikipedia](https://en.wikipedia.org/wiki/List_of_American_films_of_2010) for most popular films in the 2010s (2010-2019) and their cast.

The script isn't perfect, the data aren't perfect, as you will undoubtedly see - but they're good enough for us. For example Wikipedia only lists "main" actors on cast (A-list, sort of), for a more extensive list one could go to scrape [IMDB](https://www.imdb.com/).

Few important notes if you're interested in web scraping (if not - go on):

* You can use the script if you want to increase the span of years, e.g. to 2000-2019, but it is only guaranteed to work for 2010-2019, those 10 Wikipedia pages, on 3/3/2020 (Yes, they can change!)
* Even between 2010 and 2019 if you look closely at the script there are differences in structure
* With great power comes great responsibility - there's ethics to web scraping, please don't bombard Wikipedia or any other site with thousands of requests, they might crash or punish you and rightly so! See e.g. [here](https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01).

Read in the data, it is pretty self explanatory:

In [ ]:
films_actors_edgelist = pd.read_csv('data/films_actors_edgelist_2010s.csv', usecols=['film', 'actor', 'year'])

### Basic Data Exploration

###### (0 points)

Data dimensions:

In [ ]:
films_actors_edgelist.shape

Peek at the data:

In [ ]:
films_actors_edgelist.head(5)

Top actors (no. of films):

In [ ]:
films_actors_edgelist.groupby('actor').size().sort_values(ascending=False).head(10)

Notice the only actress in this group is Kristen Wiig. She had a film released every year excluding 2015 and 2018:

In [ ]:
films_actors_edgelist.query('actor == "Kristen Wiig"').sort_values('year')

But these are extremes. You can see that over 50% of the 4,742 actors in the list have only 1 film:

In [ ]:
films_actors_edgelist.groupby('actor').size().describe()

See top films (size of cast):

In [ ]:
films_actors_edgelist.groupby('film').size().sort_values(ascending=False).head(10)

Anchorman 2 has a cast of 32 A list actors?

In [ ]:
films_actors_edgelist.loc[films_actors_edgelist['film'].str.startswith("Anchorman 2"), 'actor'].to_list()

But these are extremes. You can see a film on this list typically has up to 7 named actors:

In [ ]:
films_actors_edgelist.groupby('film').size().describe()

### Creating the Network

##### (20 points)

Your task is to create the network of actors.

* Call it `A`
* Your network should be undirected and unweighted, i.e. if two actors were cast in at least one film together they are connected by an edge
* No self edges obviously, no multi-edges
* Be careful not to "lose" an actor/actress who was cast in a single film in which he/she are the **only** cast members
* There are numerous ways to do this, I provided hints, **YOU ONLY NEED TO IMPLEMENT ONE**

In [ ]:
# Some sanity check function to help you:
def print_network_info(G):
    print('no. of nodes: %d' % G.number_of_nodes())
    print('no. of edges: %d' % G.number_of_edges())
    print('no. of James Franco neighbors: %d' % len(list(G.neighbors('James Franco'))))

In [ ]:
# method 1: converting our DF into a <film --> actors list> dictionary, then iterating through that dictionary,
# adding an edge between every combination of 2 actors
#
# ### YOUR CODE HERE
# print_network_info(A)

In [ ]:
# method 2: use NetworkX bipartite.projected_graph() function
#
# ### YOUR CODE HERE
# print_network_info(A)

In [ ]:
# method 3 (slowest and least recommended):
# convert our DF into a films X actors 0/1 incidence matrix (e.g. with pivot_table()),
# then either:
# (1) compute the projected actors co-occurence matrix manually, then use NetworkX from_numpy_array() reading function
# (2) use NetworkX bipartite.from_biadjacency_matrix() function, then bipartite.projected_graph()
#
# ### YOUR CODE HERE
# print_network_info(A)

### Basic Network Exploration

##### (20 points)

What is the average, median minimum and maximum degree of the network?

In [ ]:
# A_degrees = ### YOUR CODE HERE ###
# print('Mean degree: %.2f' % np.mean(A_degrees))
# print('Median degree: %.2f' % np.median(A_degrees))
# print('Max degree: %.2f' % np.max(A_degrees))
# print('Min degree: %.2f' % np.min(A_degrees))

Who are the 10 actors with highest degrees?

In [ ]:
### YOUR CODE HERE ###

Plot the degree histogram of the network (doesn't have to be pretty).

In [ ]:
### YOUR CODE HERE ###

Plot the network (doesn't have to be pretty).

In [ ]:
### YOUR CODE HERE ###

Check if the network is connected:

In [ ]:
### YOUR CODE HERE ###

How many components are there?

In [ ]:
### YOUR CODE HERE

What is the size of the largest component?

In [ ]:
### YOUR CODE HERE ###

Plot the largest component (doesn't have to be pretty).

In [ ]:
### YOUR CODE HERE ###

### James Franco Number

##### (15 points)

In the 1990s they were interested in the [Kevin Bacon Number](https://simple.wikipedia.org/wiki/Bacon_number) which on a graph like ours would be the length of the shortest path between a given actress and Kevin Bacon. Also see [The Oracle of Bacon](https://oracleofbacon.org/how.php).

Let's do the same with the first actor on our list: James Franco!

Use `nx.shortest_path_length` to calculate all James Franco numbers in our graph. Notice NetworkX will calculate this only for James Franco's component, which is good enough for us, but think about what would be the shortest path to unconnected actors and what does this mean about the average you will be getting below.

In [ ]:
# james_franco_number = ### YOUR CODE HERE ###

What is the average, median, minimum and maximum James Franco number?

In [ ]:
### YOUR CODE HERE ###

What is the percentage of actors having James Franco Number of 3 or less?

In [ ]:
### YOUR CODE HERE ###

### Centrality

##### (15 points)

Compute the Degree, Eigen, Pagerank, Closeness and Betweenness centralities (may take some time):

In [ ]:
# cent_deg = ### YOUR CODE HERE ###
# cent_eig = ### YOUR CODE HERE ###
# cent_pagerank = ### YOUR CODE HERE ###
# cent_closeness = ### YOUR CODE HERE ###
# cent_betweenness = ### YOUR CODE HERE ###

Run this chunk to create a DataFrame with 5 columns, one for every centrality measure, and an `actor` column:

In [ ]:
cent_df = pd.DataFrame([cent_deg, cent_eig, cent_pagerank, cent_closeness, cent_betweenness]).T.reset_index()
cent_df.columns = ['actor', 'degree', 'eigen', 'pagerank', 'closeness', 'betweenness']
cent_df[cent_df['actor'].isin(['Meryl Streep', 'Liam Neeson', 'James Franco', 'Cate Blanchett'])]

Use `cent_df` to print the top 5 central actors for every centrality measure:

In [ ]:
for cent in cent_df.columns[1:]:
    print('Centrality %s:' % cent)
    print(cent_df.sort_values(cent, ascending=False)['actor'].head(5))
    print()

**Do you see the actor which is in the top 5 centrality by any measure? Look at this actor's films list and write why he's so central.**

In [ ]:
### YOUR ANSWER HERE ###

### Community Detection

##### (15 points)

Use the Louvain method to detect communities of actors in **the giant component** of our network. Use `seed=42`.

In [ ]:
# communities = ### YOUR CODE HERE

What is the modularity of the partition found by the algorithm?

In [ ]:
### YOUR CODE HERE ###

How many communities were found by the Louvain method? What is the average, median, minimum and maximum community size?

In [ ]:
### YOUR CODE HERE ###

In order to make some sense of the community structure found by the Louvain method, let us filter for each community only A-list actors, e.g. actors with at least 5 films in the decade. Also sort the actors alphabetically in each list:

In [ ]:
actor2n_films = films_actors_edgelist.groupby('actor').size().to_dict()
a_list = [actor for actor, n_films in actor2n_films.items() if n_films >= 5]
print('Found %d actors with at least 5 films' % len(a_list))
communities_a_list = ### YOUR CODE HERE ###

For each community filtered print its actors:

In [ ]:
for actors in communities_a_list:
    if len(actors) > 10:
        print(actors)
        print()

It's extremely hard to make sense of the communities and some of them don't really make sense, but if you followed thus far, you can see a few interesting ones.

Write a function which takes an A-list actor's name, and returns the (A-list) community to which this actor belongs (or an empty list of the actor was not found):

In [ ]:
# def get_actor_community(actor):
#     ### YOUR CODE HERE ###

The Scarlett Johansson community looks like a "blockbuster" community, with all the Marvel stars (Chris Hemsworth, Chris Prat, Benedict Cumberbatch, Chadwick Boseman, Brie Larson, Idris Elba, Mark Ruffalo etc.).

In [ ]:
get_actor_community('Scarlett Johansson')

The Tina Fey community looks like a "softer" films community, it includes Amy Poehler, Jim Carrey, Paul Rudd, Will Ferrell, etc.

In [ ]:
get_actor_community('Tina Fey')

The Helena Bonham Carter community looks like the "Brits" community, it includes: Alan Rickman, Emily Blunt, Hugh Laurie, Emma Watson, Daniel Radcliffe, Ralph Fiennes, Maggie Smith etc.

In [ ]:
get_actor_community('Helena Bonham Carter')

**Not all communities make sense**

For example Daisy Ridley's community incorporates her Star Wars colleagues (Mark Hamill, John Boyega, Adam Driver), but also... Jenna Fischer and Laura Dern.

In [ ]:
get_actor_community('Daisy Ridley')

### Paper questions

##### (15 points)

Read Overgoor et. al. [Structure of U.S. College Networks on Facebook](https://ojs.aaai.org/index.php/ICWSM/article/view/7318) AAAI 2020 paper, first 6 pages (of course you're invited to read the whole thing!)

What is the density of the US college networks graph (it's a simple computation)?

In [ ]:
### YOUR CODE HERE ###

In your own up to 100 words, what is described in Figure 6? Copy pasting from the paper will lead to zero points.

In [ ]:
### YOUR ANSWER HERE ###

### Wrap up
And that's it, you have shown you can use NetworkX to build interesting networks, find some interesting facts and metrics about them and run community detection algorithms. Good luck with the rest of the course!